In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
import os
os.kill(os.getpid(), 9)

In [50]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.1 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125924 sha256=6ff4e6458f0a058690044e3d15f906a1694977758e1ccad55494df682cb282d6
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 4.7 MB/s eta 0:00:00


In [3]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.4 MB/s eta 0:00:00


In [4]:
!pip install -i https://test.pypi.org/simple/ bitsandbytes

Looking in indexes: https://test.pypi.org/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 MB 11.5 MB/s eta 0:00:00


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig


In [127]:
class Prompt:
  def technical_feedback_prompt(self, product_problem):
    return """
      You are a full stack web developer and your role is to give technical feedback to your product manager who will give you a problem about the product

      You have to analyze the problem and give your thoughts on how to implement or address this issue from the engineering perspective.

      You may give an explanation of technologies that are needed, how many resources are needed and what in a high-level needs to be done to address the product concerns.

      Here is the product manager's problem below:
      %s

      You must respond with your full technical feedback for the problem above and address the product manager's issue.
    """ % (product_problem)


  def ticket_sizing_prompt(self, ticket_description, sizes):
    return """
      You are a full stack web developer and your role is to size tickets that given to you.
      "Ticket descriptions" define a problem/task for you, the developer, to address
      "Ticket sizing" is when you assign a ticket with a rating based on how complex/difficult the task is.
      Your reasonings for sizing this ticket must be related to the complexity of the issue and your ability as a "junior" developer to solve it

      Using your knowledge of full stack web developing size the ticket using one of the following numbers: %s

      Each size value increases the complexity by a factor

      Here's the ticket description below:
      %s

      You must only respond with the ticket size you chose (only using the sizes %s) and an explaination of why you chose this ticket size
    """ % (sizes, ticket_description, sizes)

  def bug_fix_prompt(self, codes, manager_problem):
    return """
      You are a full stack web developer and your role is to fix bugs given the affected files. Some of the files might not be affected. Use your judgement to determine which files need to be fixed. And rewrite the full fixed file(s).

      Below is the bug your manager is telling you need to fix:
      %s


      The affected code files you must fix are below.
      %s

      Only respond with the NEW full fixed code files in markdown format. Do not respond with a reiteration of the files you were given just the new fixed files including the older code but with your new additions. Also do not respond with any of the explanations about how you fixed the files.
    """ % (manager_problem, codes)

  def test_prompt(self, code, test_framework):
    return """
      You are a full stack web developer and your role is to write tests for files that are given to you.

      You must use the %s framework to implement the test.

      Only respond with the written test in markdown format. Do not respond with any of the explanations about how you implemented it

      The code you must write a test for is below:

      CODE: %s
    """ % (test_framework, code)

  def bootstrap_project_prompt(self, technologies):
    return """
You are a full stack web developer and your role is to setup a coding project from scratch. You must use include ALL the technologies below:

%s

Think step by step and reason yourself to the right decisions to make sure we get it right.
First lay out the names of the core classes, functions, methods that will be necessary, As well as a quick comment on their purpose.

Then you will output the content of each file including ALL code.
Each file must strictly follow a markdown code block format, where the following tokens must be replaced such that
FILENAME is the lowercase file name including the file extension,
LANG is the markup code block language for the code's language, and CODE is the code:

**FILENAME**
```LANG
CODE
```

Do not comment on what every file does. Please note that the code should be fully functional. No placeholders.

You will start with the "entrypoint" file, then go to the ones that are imported by that file, and so on.
Please note that the code should be fully functional. No placeholders.

Follow a language and framework appropriate best practice file naming convention.
Make sure that files contain all imports, types etc.  The code should be fully functional. Make sure that code in different files are compatible with each other.
Ensure to implement all code, if you are unsure, write a plausible implementation.
Include module dependency or package manager dependency definition file.
Before you finish, double check that all parts of the architecture is present in the files.

    """ % technologies

In [20]:
model_name="WizardLM/WizardCoder-15B-V1.0"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map={"": Accelerator().process_index},
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [129]:
import torch
from accelerate import Accelerator

class CoderLLM:
  def __init__(self, prompt, model_name="bigcode/starcoder"):
    self.model_name = model_name
    self.load_8bit = True
    self.model = None;
    self.tokenizer = None;
    self.prompt = prompt
    self.base_prompt = """
    Below is an instruction that describes a task. Write a response that appropriately completes the request.

    ### Instruction:
    %s

    ### Response:
    """

  def load(self):
    self.model = model
    self.tokenizer = tokenizer

  def get_instruction_prompt(self, instruction):
    return self.base_prompt % instruction

  def get_write_test_prompt(self, code, test_framework):
    return self.get_instruction_prompt(self.prompt.test_prompt(code, test_framework))

  def get_bootstrap_project_prompt(self, technologies):
    return self.get_instruction_prompt(self.prompt.bootstrap_project_prompt(technologies))

  def get_fix_bug_prompt(self, codes, manager_problem):
    return self.get_instruction_prompt(self.prompt.bug_fix_prompt(codes, manager_problem))

  def get_ticket_sizing_prompt(self, ticket_description, sizes):
    return self.get_instruction_prompt(self.prompt.ticket_sizing_prompt(ticket_description, sizes))

  def get_technical_feedback_prompt(self, product_problem):
    return self.get_instruction_prompt(self.prompt.technical_feedback_prompt(product_problem))

  def generate(self, instruction,
        input=None,
        temperature=1,
        top_p=0.9,
        top_k=40,
        num_beams=1,
        max_new_tokens=2048,
        **kwargs,):
    inputs = self.tokenizer(instruction, return_tensors="pt")
    input_ids = inputs["input_ids"].to("cuda")
    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        do_sample=True,
        eos_token_id=self.tokenizer.eos_token_id,
        pad_token_id=self.tokenizer.pad_token_id,
        **kwargs,
    )
    with torch.no_grad():
        generation_output = self.model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=max_new_tokens,
        )
    s = generation_output.sequences
    output = self.tokenizer.batch_decode(s, skip_special_tokens=True)
    return output


In [130]:
prompt = Prompt()

In [131]:
print("Loading the LLM:", model_name)
coder_llm = CoderLLM(prompt, model_name=model_name)
coder_llm.load()

Loading the LLM: WizardLM/WizardCoder-15B-V1.0


In [10]:
!git clone https://github.com/jawerty/threadwatch.xyz.git

Cloning into 'threadwatch.xyz'...
remote: Enumerating objects: 343, done.
remote: Total 343 (delta 0), reused 0 (delta 0), pack-reused 343
Receiving objects: 100% (343/343), 1.93 MiB | 27.79 MiB/s, done.
Resolving deltas: 100% (205/205), done.


In [16]:
!ls threadwatch.xyz/server/frontend/components

App.js		  Header.js		OnboardingFlow.js  Thread.js
CommenterView.js  LinkGeneratorForm.js	ReactionVoter.js   ThreadList.js
FrontPage.js	  Loading.js		ThreadCard.js	   TopicSelector.js


In [126]:
from sentence_transformers import SentenceTransformer, util

import re
import glob
import pprint

pp = pprint.PrettyPrinter(indent=4)

class JuniorDev:
  def __init__(self, coder_llm):
    self.coder_llm = coder_llm
    self.sentence_transformers = SentenceTransformer("multi-qa-mpnet-base-dot-v1")

  def write_test(self, filename, test_framework="jest"):
    print("Writing test for", filename)
    with open(filename, 'r', encoding='utf-8') as f:
      lines = f.readlines()
      code = "".join(lines)
      code = code[:4096]

    prompt = self.coder_llm.get_write_test_prompt(code, test_framework)

    output = self.coder_llm.generate(prompt)[0]

    output = output[output.index("### Response:"):]
    test_code = re.findall(r"```(.*?)```", output, re.DOTALL)[0]
    return test_code
    # generate here

  def bootstrap_project(self, technologies):
    technologies = "\n".join([f"- {technology}" for technology in technologies])

    prompt = self.coder_llm.get_bootstrap_project_prompt(technologies)

    output = self.coder_llm.generate(prompt, max_new_tokens=8000)[0]

    output = output[output.index("### Response:"):]
    print(output)
    code_blocks = re.findall(r"```(.*?)```", output, re.DOTALL)
    file_names = re.findall(r"\*\*(.*?)\*\*", output, re.DOTALL)
    code_files = []
    for i in range(0, len(file_names)):
      if i < len(code_blocks):
        code_files.append({
            "file_name": file_names[i],
            "code_block": code_blocks[i]
        })

    for file_code_pair in code_files:
      filepath = "bootstrapped-project/"+file_code_pair["file_name"]
      os.makedirs(os.path.dirname(filepath), exist_ok=True)
      with open(filepath, "w+") as f:
        code_block = file_code_pair["code_block"].split("\n")[1:]
        f.write("\n".join(code_block).encode('ascii', 'ignore').decode('ascii'))


    # code_blocks = re.findall(r"```(.*?)```", output, re.DOTALL)

    # print(code_blocks)

  def fix_bug(self, folder, manager_problem):
    code_embeddings = []

    ignore_folders = ["node_modules/", "env/"]
    ignore_files = [".json", ".txt", ".md"]

    for filename in glob.iglob(f"./{folder}/**/*", recursive=True):
      ignore_folder = len(list(filter(lambda x: x in filename, ignore_folders))) > 0
      ignore_file = len(list(filter(lambda x: x in filename, ignore_files))) > 0
      is_a_file = "." in filename.split("/")[-1]
      if ignore_folder or ignore_file or not is_a_file:
        print("ignore", filename)
        continue

      print("Embeddings for", filename)
      with open(filename, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        code = "".join(lines)
        code = code[:4096]

      embedding = self.sentence_transformers.encode(f"{filename}\n{code}")
      code_embeddings.append({
          "embedding": embedding,
          "filename": filename,
          "code": f"New File: {filename}\n{code}"
      })

    query = self.sentence_transformers.encode(manager_problem)

    new_embeddings = []
    for embed in code_embeddings:
      embed["score"] = util.dot_score(query, embed["embedding"]).item()
      new_embeddings.append(embed)

    del code_embeddings

    top_files = sorted(new_embeddings, key=lambda x: x["score"], reverse=True)[:3]
    print("Affected files")
    for _file in top_files:
      print(_file["filename"])

    top_matching_files = "\n".join(list(map(lambda x: x["code"], top_files)))

    prompt = self.coder_llm.get_fix_bug_prompt(top_matching_files, manager_problem)
    output = self.coder_llm.generate(prompt, max_new_tokens=4096)[0]
    output = output[output.index("### Response:"):]
    print(output)

  def size_ticket(self, ticket_desription, sizes=[1,2,3,5,8]):
    prompt = self.coder_llm.get_ticket_sizing_prompt(ticket_desription, ", ".join([str(x) for x in sizes]))
    output = self.coder_llm.generate(prompt, max_new_tokens=4096)[0]
    output = output[output.index("### Response:"):]
    print(output)
    # vecorize code
    # for every file in the codebase vectorize all the files and fine the file that matches the problem the most

  def technical_feedback(self, product_problem):
    prompt = self.coder_llm.get_technical_feedback_prompt(product_problem)
    output = self.coder_llm.generate(prompt, max_new_tokens=2048)[0]
    output = output[output.index("### Response:"):]
    print(output)

In [133]:
junior_dev = JuniorDev(coder_llm)
# test_code = junior_dev.write_test("./threadwatch.xyz/server/frontend/components/LinkGeneratorForm.js", test_framework="enzyme")

# print("\n\nFinal test:\n\n")
# print(test_code)

# output = junior_dev.bootstrap_project(["react", "jest", "webpack", "tailwind"])

# output = junior_dev.fix_bug("./threadwatch.xyz", "The header file is styled weirdly. I want all the css to be using flex for positioning")

junior_dev.technical_feedback("As a product manager for an ecommerce website. I want to have an interal tool that is a browser extension that allows me to see the full database information for products that I see on the product page. How can we do this?")

### Response:
    
      1. The issue is that the internal tool is a browser extension, but the product manager wants to have it for mobile devices as well. So, it is not feasible to create a browser extension for mobile devices. Instead, they need to create a mobile app or integrate a mobile SDK with their existing website.
       
      2. To address this, the product manager can follow these steps:
          a. Research mobile ecommerce solutions, such as Shopify, WooCommerce, and Magento Mobile.
          b. Choose a mobile ecommerce solution and integrate it with their website.
          c. Implement the internal tool as a mobile feature that allows users to see full product details.
          
      3. To implement the internal tool, the product manager can use a frontend framework such as React Native, Ionic, or Flutter. They also need to research and choose a database for storing product information.
       
      4. Finally, the product manager can create a user interface for 